In [3]:
"""
Rewriting the methods given for the conversion to JzAzBz Space. A direct translation of the code given by 
https://figshare.com/articles/software/JzAzBz_m/5016299

Later notebooks use the SimpleJzAzBz Class to conduct analysis
"""

import numpy as np
import math

In [4]:
#Array for XYZ to LMS conversion
M1 = np.array([[0.41478972, 0.579999, 0.014648],
                  [-0.20151, 1.120649, 0.0531008],
                  [-0.0166008, 0.2648, 0.6684799]])

#Array for L'M'S' to IzAzBz 
M2 = np.array([[0.5, 0.5, 0],
                [3.524000,  -4.066708, 0.542708],
                [0.199076, 1.096799,  -1.295875]])

#Constants
b=1.15
g=0.66
d = -0.56
n = 2610 / (2**14)
p = (1.7) * 2523/ (2**5) 
c1 = 3424/ (2**12)
c2 = 2413/ (2**7) 
c3 = 2392/ (2**7)

#Perceptual Quantizer -> curve which is a dynamic transform of cone responses.
def PQ(X, lum=10000):
    
    #Note: I am pretty sure that the 10000 value comes from the luminance value of 10000 cd/m2, candelas per square metre.
    # It is possible that this value can be adjusted for varying luminances. As such I've made it a parameter that can
    # be changed if one desires.
    xn = np.sign(X / lum) * (np.abs(X / lum) ** n) #(x./10000).^n
    a = c1 + (c2 * xn)
    b = 1 + (c3 * xn)
    
    LMS_prime = np.sign(a/b) * (np.abs(a/b) ** p)
    return LMS_prime

#Conversion from XYZ to IzAzBz
def xyz_2_izazbz(XYZ_D65):
    #XYZ_D65 is an n by 3 matrix
    XYZ_D65[:,1] = g * XYZ_D65[:,1] - np.multiply((g-1),XYZ_D65[:,0])
    XYZ_D65[:,0] = b * XYZ_D65[:,0] - np.multiply((b-1),XYZ_D65[:,2])

    LMS_prime = PQ(XYZ_D65 @ M1.T)
    IzAzBz = LMS_prime @ M2.T 

    return IzAzBz

#Conversion of Iz to Jz
def iz_2_jz(IzAzBz):
    Iz = IzAzBz[:,0]
    Jz = ((1+d) * Iz) / (1+d*Iz)-1.6295499532821566e-11
    JzAzBz =  np.array([Jz, IzAzBz[:,1], IzAzBz[:,2]]).T 
    
    return JzAzBz

#Putting it all together in xyz -> jzazbz
def xyz_2_jzazbz(XYZ_D65):
    #convinient function

    XYZ_D65[:,1] = g * XYZ_D65[:,1] - np.multiply((g-1),XYZ_D65[:,0])
    XYZ_D65[:,0] = b * XYZ_D65[:,0] - np.multiply((b-1),XYZ_D65[:,2])

    LMS_prime = PQ(XYZ_D65 @ M1.T)
    IzAzBz = LMS_prime @ M2.T  
    
    Iz = IzAzBz[:,0]
    Jz = ((1+d) * Iz) / (1+d*Iz)-1.6295499532821566e-11
    JzAzBz =  np.array([Jz, IzAzBz[:,1], IzAzBz[:,2]]).T 
    
    return JzAzBz


In [5]:
# Test array with expected output values below, for XYZ to JzAzBz

test = np.array([[0.20654008, 0.12197225, 0.05136952]]) #xyz
izazbz = xyz_2_izazbz(test)
print('Izazbz out:', izazbz)
print('Izazbz test:''[ 0.0120779...,  0.0092430...,  0.0052600...]','\n')

jzazbz = iz_2_jz(izazbz)
print('Jzazbz out:', jzazbz)
print('JzAzBz test:', '[ 0.0053504...,  0.0092430...,  0.0052600...]\n')

test = np.array([[0.20654008, 0.12197225, 0.05136952]]) #xyz
jzazbz2 = xyz_2_jzazbz(test)
print('Jzazbz out:', jzazbz2)
print('JzAzBz test:', '[ 0.0053504...,  0.0092430...,  0.0052600...]')

Izazbz out: [[0.01207793 0.00924302 0.00526007]]
Izazbz test:[ 0.0120779...,  0.0092430...,  0.0052600...] 

Jzazbz out: [[0.00535048 0.00924302 0.00526007]]
JzAzBz test: [ 0.0053504...,  0.0092430...,  0.0052600...]

Jzazbz out: [[0.00535048 0.00924302 0.00526007]]
JzAzBz test: [ 0.0053504...,  0.0092430...,  0.0052600...]


In [6]:
#An inverse construction of the function for JzAzBz -> XYZ
def invPQ(X):
    #Converts L'M'S' into LMS
    xn = np.sign(X) * (np.abs(X)**(1/p)) #x.^(1/p)
    a = c1-xn
    b = c3*xn-c2
    
    LMS = 10000 * ( np.sign(a/b) * (np.abs(a/b)**(1/n)))
    
    return LMS
    
    
def jzazbz_2_xyz(JzAzBz):
    #Takes an nx3 array as input.
    
    JzAzBz[:,0] = JzAzBz[:,0]+1.6295499532821566e-11
    Iz = JzAzBz[:,0] / (1 + d - (d*JzAzBz[:,0]))
    
    IzAzBz = np.array([Iz, JzAzBz[:,1], JzAzBz[:,2]]).T

    XYZ_D65 = invPQ(IzAzBz @ np.linalg.inv(M2).T) @ np.linalg.inv(M1).T
    
    XYZ_D65[:,0] = (XYZ_D65[:,0] + np.multiply((b-1),XYZ_D65[:,2])) / b 
    XYZ_D65[:,1] = (XYZ_D65[:,1] + np.multiply((g-1),XYZ_D65[:,0])) / g

    return XYZ_D65

In [7]:
#Testing JzAzBz to XYZ

out = jzazbz_2_xyz(jzazbz)
print('xyz out:', out)
print('original xyz:', '[[0.20654008, 0.12197225, 0.05136952]]')

xyz out: [[0.20654008 0.12197225 0.05136952]]
original xyz: [[0.20654008, 0.12197225, 0.05136952]]


In [8]:
#Formulas for converting JzAzBz to JzCzHz and deltaE

def jzazbz_2_jzczhz(JzAzBz):
    #input is an nx3 array
    #cz = sqrt(az^2 + bz^2)
    Cz = np.array([math.sqrt( (JzAzBz[:,1]**2) + (JzAzBz[:,2]**2) )])
    
    #hz = arctan2(bz/az)
    Hz = np.arctan2(JzAzBz[:,2], JzAzBz[:,1])
    
    return np.array([JzAzBz[:,0], Cz, Hz]).T


def calculate_deltaE(JzAzBz1, JzAzBz2):
    #Delta E = sqrt(deltaj^2 + deltac^2 + deltah^2)
    #delta h = 2 * sqrt(c2*c1) * sin(deltah/2)
    
    JzCzHz1 = jzazbz_2_jzczhz(JzAzBz1)
    JzCzHz2 = jzazbz_2_jzczhz(JzAzBz2)
    
    d_j = JzCzHz2[:,0] - JzCzHz1[:,0]
    d_c = JzCzHz2[:,1] - JzCzHz1[:,1]
    
    d_h = 2 * math.sqrt(JzCzHz2[:,1] * JzCzHz1[:,1]) * np.sin( (JzCzHz2[:,2] - JzCzHz1[:,2]) / 2 ) 
    
    return np.sqrt( (d_j**2) + (d_c**2) + (d_h**2))


In [9]:
#Testing DeltaE with expected values

#345 triangle, expected out is 0.05
c1 = np.array([[0.1, 0.03, 0.04]]) #jzazbz
c2 = np.array([[0.1, 0.0, 0.0]])

de = calculate_deltaE(c1, c2)
print('DeltaE', de, 'expected', '0.05')

DeltaE [0.05] expected 0.05


In [57]:
#Function for transforming XYZ to sRGB

#conversion array given here: http://www.brucelindbloom.com
#conversion from XYZ65 to RGB
M3 = np.array([[3.2404542, -1.5371385, -0.4985314],
                [-0.9692660,  1.8760108,  0.0415560],
                [0.0556434, -0.2040259,  1.0572252]])

#Conversion from sRGB to XYZ65
M4 = np.array([[0.4124564, 0.3575761, 0.1804375],
                 [0.2126729, 0.7151522, 0.0721750],
                 [0.0193339, 0.1191920, 0.9503041]])

def xyz65_2_srgb(XYZ_D65):
    rgb = XYZ_D65 @ M3.T
    
    rgb[:,0] = _nonlinear_xyz_rgb(rgb[:,0])
    rgb[:,1] = _nonlinear_xyz_rgb(rgb[:,1])
    rgb[:,2] = _nonlinear_xyz_rgb(rgb[:,2])
    
    return rgb * 255

def srgb_2_xyz65(sRGB):
    sRGB = sRGB / 255
    
    sRGB[:,0] = _nonlinear_rgb_xyz(sRGB[:,0])
    sRGB[:,1] = _nonlinear_rgb_xyz(sRGB[:,1])
    sRGB[:,2] = _nonlinear_rgb_xyz(sRGB[:,2])
    
    xyz65 = (M4 @ sRGB.T).T
    return xyz65

def _nonlinear_xyz_rgb(x):
    if x <= 0.0031308:
        return x * 12.92
    else:
        return 1.055 * (x ** (1/2.4)) - 0.055
    
def _nonlinear_rgb_xyz(x):
    if x <= 0.04045:
        return x / 12.92
    else:
        return (np.sign((x+0.055)/1.055) * np.power(np.abs(x+0.055)/1.055, 2.4))

In [58]:
#Testing XYZ white 
white_test = np.array([[1.0,1.0,1.0]]) #white in XYZ 65
white = xyz65_2_srgb(white_test)

print('rgb white', white)
print('1.085157, 0.976922, 0.958809')

test2 = np.array([[255.0,255.0,255.0]])
out = srgb_2_xyz65(test2)
print(out)

rgb white [[276.71504155 249.11499097 244.49626643]]
1.085157, 0.976922, 0.958809
[[0.95047   1.0000001 1.08883  ]]
